In [2]:
import pandas as pd
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import transformers
import sentence_transformers
from transformers import AutoModel, AutoTokenizer
from transformer_helper import get_token_counts, get_embeddings
from data_helper import select_first_n_occurences, main, main_2_pos_2_neg
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
col = "StoryBody"
df = main()
df0 = df.copy()
df_bis = main_2_pos_2_neg()

20 rows
30 rows


In [4]:
model_path = 'Alibaba-NLP/gte-base-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).to(device)

# First example

In [11]:
df, _ = select_first_n_occurences(df, n=2)
pd.concat((df0, df), axis=1, keys=["df", "df_max_3_occ"])

df         \
                              StoryBody id_qis   
0                   This is quite rare.    id2   
1              This stock is overrated.    id1   
2          The CEO is completely crazy.    id1   
3      I would like to buy their items.    id2   
4              This stock is overrated.    id1   
5               What a surprising deal.    id1   
6                   This is quite rare.    id2   
7                     I love this firm.    id2   
8            I doubt they will make it.    id3   
9                   This is quite rare.    id2   
10          This has so much potential.    id3   
11  This new journey seems interesting.    id3   
12         The CEO is completely crazy.    id1   
13              What a surprising deal.    id1   
14              What a surprising deal.    id1   
15          This has so much potential.    id3   
16                  This is quite rare.    id2   
17           I doubt they will make it.    id3   
18           I doubt they will make it.    id3   
19     I would like to buy their items.    id2   

                           df_max_3_occ         
                              StoryBody id_qis  
0                   This is quite rare.    id2  
1              This stock is overrated.    id1  
2          The CEO is completely crazy.    id1  
3      I would like to buy their items.    id2  
4              This stock is overrated.    id1  
5               What a surprising deal.    id1  
6                   This is quite rare.    id2  
7                     I love this firm.    id2  
8            I doubt they will make it.    id3  
9                                   NaN    NaN  
10          This has so much potential.    id3  
11  This new journey seems interesting.    id3  
12         The CEO is completely crazy.    id1  
13              What a surprising deal.    id1  
14                                  NaN    NaN  
15          This has so much potential.    id3  
16                                  NaN    NaN  
17           I doubt they will make it.    id3  
18                                  NaN    NaN  
19     I would like to buy their items.    id2

In [5]:
HARD_LIMIT = 34_002
SOFT_LIMIT = 8192
token_counts = {}
embeddings = {}
cctnd_contents = {}
for id_qi in df["id_qis"].unique():
    df_id_qi = df[df["id_qis"]==id_qi]
    cctnd_content = " ".join(df_id_qi[col].to_list())
    cctnd_contents[id_qi] = cctnd_content
    token_counts[id_qi] = get_token_counts([cctnd_content], tokenizer)[0]
    assert token_counts[id_qi] <= HARD_LIMIT
    if token_counts[id_qi] >= SOFT_LIMIT:
        print(f"ID_QI {id_qi}: {token_counts[id_qi]} tokens (max. recommended is {SOFT_LIMIT})")

In [6]:
id_qis = list(cctnd_contents.keys())
embeddings = get_embeddings(list(cctnd_contents.values()), model, tokenizer, device).cpu()
embeddings = pd.DataFrame(embeddings, index=id_qis)

C:\Users\Tim\.cache\huggingface\modules\transformers_modules\Alibaba-NLP\new-impl\e244c9398934a48b4663dacf7ff4965961a6807a\modeling.py:579: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
pd.DataFrame(cosine_similarity(embeddings), index=id_qis, columns=id_qis)

,id2,id1,id3
id2,1.000000,0.412259,0.367724
id1,0.412259,1.000000,0.358449
id3,0.367724,0.358449,1.000000


In [8]:
for id_qi, content in cctnd_contents.items():
    print(f"{id_qi}: {content}")

id2: This is quite rare. I would like to buy their items. This is quite rare. I love this firm. This is quite rare. This is quite rare. I would like to buy their items.
id1: This stock is overrated. The CEO is completely crazy. This stock is overrated. What a surprising deal. The CEO is completely crazy. What a surprising deal. What a surprising deal.
id3: I doubt they will make it. This has so much potential. This new journey seems interesting. This has so much potential. I doubt they will make it. I doubt they will make it.


# Second example

In [9]:
df_bis, _ = select_first_n_occurences(df_bis, n=2)

In [10]:
HARD_LIMIT = 34_002
SOFT_LIMIT = 8192
token_counts = {}
embeddings = {}
cctnd_contents = {}
for id_qi in df_bis["id_qis"].unique():
    df_id_qi = df_bis[df_bis["id_qis"]==id_qi]
    cctnd_content = " ".join(df_id_qi[col].to_list())
    cctnd_contents[id_qi] = cctnd_content
    token_counts[id_qi] = get_token_counts([cctnd_content], tokenizer)[0]
    assert token_counts[id_qi] <= HARD_LIMIT
    if token_counts[id_qi] >= SOFT_LIMIT:
        print(f"ID_QI {id_qi}: {token_counts[id_qi]} tokens (max. recommended is {SOFT_LIMIT})")

In [11]:
id_qis = list(cctnd_contents.keys())
embeddings = get_embeddings(list(cctnd_contents.values()), model, tokenizer, device).cpu()
embeddings = pd.DataFrame(embeddings, index=id_qis)

In [12]:
pd.DataFrame(cosine_similarity(embeddings), index=id_qis, columns=id_qis)

,id3,id2,id1,id5
id3,1.000000,0.513939,0.560053,0.226419
id2,0.513939,1.000000,0.306629,0.235084
id1,0.560053,0.306629,1.000000,0.345450
id5,0.226419,0.235084,0.345450,1.000000


In [13]:
# the fact that cos(i1, i3) = 0.56 is so high even though the sentiment is opposed shows that the embeddings do not convey sentiment
# i will need to extract the sentiment one way or the other
for id_qi, content in cctnd_contents.items():
    print(f"{id_qi}: {content}")

id3: Excellent. This is interesting. This is interesting. Excellent. Can't wait to see their new CEO! Can't wait to see their new CEO!
id2: This is quite rare. I love this firm. I love this firm. I would like to buy their items.
id1: The CEO is completely crazy. This stock is overrated. This stock is overrated. The CEO is completely crazy. This is a scandal.
id5: i dislike it so much. i dislike it so much. It deserves to be ignored. It deserves to be ignored.
